In [4]:
!pip install ml_metrics

  Running setup.py install for ml-metrics: started
    Running setup.py install for ml-metrics: finished with status 'done'


In [5]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import math 
np.random.seed(2019)
from scipy.stats import skew
from scipy import stats
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from xgboost.sklearn import XGBClassifier
from xgboost.sklearn import XGBRegressor

import statsmodels

#!pip install ml_metrics
from ml_metrics import rmsle

import matplotlib.pyplot as plt
%matplotlib inline
print("done")

done


In [6]:
def read_and_concat_dataset(training_path, test_path):
    train = pd.read_csv(training_path)
    train['train'] = 1
    test = pd.read_csv(test_path)
    test['train'] = 0
    data = train.append(test, ignore_index=True)
    return train, test, data

train, test, data = read_and_concat_dataset('./input/train.csv', './input/test.csv')
data = data.set_index('Id')

In [7]:
def filling_missing_values(data,variable, new_value):
    data[variable] = data[variable].fillna(new_value)

In [8]:
filling_missing_values(data,'GarageCond','None')
filling_missing_values(data,'GarageQual','None')
filling_missing_values(data,'FireplaceQu','None')
filling_missing_values(data,'BsmtCond','None')
filling_missing_values(data,'BsmtQual','None')
filling_missing_values(data,'PoolQC','None')
filling_missing_values(data,'MiscFeature','None')

In [9]:
data['MSSubClass'][data['MSSubClass'] == 20] = '1-STORY 1946 & NEWER ALL STYLES'
data['MSSubClass'][data['MSSubClass'] == 30] = '1-STORY 1945 & OLDER'
data['MSSubClass'][data['MSSubClass'] == 40] = '1-STORY W/FINISHED ATTIC ALL AGES'
data['MSSubClass'][data['MSSubClass'] == 45] = '1-1/2 STORY - UNFINISHED ALL AGES'
data['MSSubClass'][data['MSSubClass'] == 50] = '1-1/2 STORY FINISHED ALL AGES'
data['MSSubClass'][data['MSSubClass'] == 60] = '2-STORY 1946 & NEWER'
data['MSSubClass'][data['MSSubClass'] == 70] = '2-STORY 1945 & OLDER'
data['MSSubClass'][data['MSSubClass'] == 75] = '2-1/2 STORY ALL AGES'
data['MSSubClass'][data['MSSubClass'] == 80] = 'SPLIT OR MULTI-LEVEL'
data['MSSubClass'][data['MSSubClass'] == 85] = 'SPLIT FOYER'
data['MSSubClass'][data['MSSubClass'] == 90] = 'DUPLEX - ALL STYLES AND AGES'
data['MSSubClass'][data['MSSubClass'] == 120] = '1-STORY PUD (Planned Unit Development) - 1946 & NEWER'
data['MSSubClass'][data['MSSubClass'] == 150] = '1-1/2 STORY PUD - ALL AGES'
data['MSSubClass'][data['MSSubClass'] == 160] = '2-STORY PUD - 1946 & NEWER'
data['MSSubClass'][data['MSSubClass'] == 180] = 'PUD - MULTILEVEL - INCL SPLIT LEV/FOYER'
data['MSSubClass'][data['MSSubClass'] == 190] = '2 FAMILY CONVERSION - ALL STYLES AND AGES'

In [10]:
def fixing_ordinal_variables(data, variable):
    data[variable][data[variable] == 'Ex'] = 5
    data[variable][data[variable] == 'Gd'] = 4
    data[variable][data[variable] == 'TA'] = 3
    data[variable][data[variable] == 'Fa'] = 2
    data[variable][data[variable] == 'Po'] = 1
    data[variable][data[variable] == 'None'] = 0

In [11]:
fixing_ordinal_variables(data,'ExterQual')
fixing_ordinal_variables(data,'ExterCond')
fixing_ordinal_variables(data,'BsmtCond')
fixing_ordinal_variables(data,'BsmtQual')
fixing_ordinal_variables(data,'HeatingQC')
fixing_ordinal_variables(data,'KitchenQual')
fixing_ordinal_variables(data,'FireplaceQu')
fixing_ordinal_variables(data,'GarageQual')
fixing_ordinal_variables(data,'GarageCond')
fixing_ordinal_variables(data,'PoolQC')

In [12]:
data['PavedDrive'][data['PavedDrive'] == 'Y'] = 3
data['PavedDrive'][data['PavedDrive'] == 'P'] = 2
data['PavedDrive'][data['PavedDrive'] == 'N'] = 1

In [13]:
colu = data.columns[(data.isnull().sum()<50) & (data.isnull().sum()>0)]
for i in colu:
    print(data[colu].isnull().sum())

BsmtFinSF1       1
BsmtFinSF2       1
BsmtFullBath     2
BsmtHalfBath     2
BsmtUnfSF        1
Electrical       1
Exterior1st      1
Exterior2nd      1
Functional       2
GarageArea       1
GarageCars       1
KitchenQual      1
MSZoning         4
MasVnrArea      23
MasVnrType      24
SaleType         1
TotalBsmtSF      1
Utilities        2
dtype: int64
BsmtFinSF1       1
BsmtFinSF2       1
BsmtFullBath     2
BsmtHalfBath     2
BsmtUnfSF        1
Electrical       1
Exterior1st      1
Exterior2nd      1
Functional       2
GarageArea       1
GarageCars       1
KitchenQual      1
MSZoning         4
MasVnrArea      23
MasVnrType      24
SaleType         1
TotalBsmtSF      1
Utilities        2
dtype: int64
BsmtFinSF1       1
BsmtFinSF2       1
BsmtFullBath     2
BsmtHalfBath     2
BsmtUnfSF        1
Electrical       1
Exterior1st      1
Exterior2nd      1
Functional       2
GarageArea       1
GarageCars       1
KitchenQual      1
MSZoning         4
MasVnrArea      23
MasVnrType      24
SaleT

In [14]:
colu = data.columns[data.isnull().sum()>=50]
for i in colu:
    print(data[colu].isnull().sum())

Alley           2721
BsmtExposure      82
BsmtFinType1      79
BsmtFinType2      80
Fence           2348
GarageFinish     159
GarageType       157
GarageYrBlt      159
LotFrontage      486
SalePrice       1459
dtype: int64
Alley           2721
BsmtExposure      82
BsmtFinType1      79
BsmtFinType2      80
Fence           2348
GarageFinish     159
GarageType       157
GarageYrBlt      159
LotFrontage      486
SalePrice       1459
dtype: int64
Alley           2721
BsmtExposure      82
BsmtFinType1      79
BsmtFinType2      80
Fence           2348
GarageFinish     159
GarageType       157
GarageYrBlt      159
LotFrontage      486
SalePrice       1459
dtype: int64
Alley           2721
BsmtExposure      82
BsmtFinType1      79
BsmtFinType2      80
Fence           2348
GarageFinish     159
GarageType       157
GarageYrBlt      159
LotFrontage      486
SalePrice       1459
dtype: int64
Alley           2721
BsmtExposure      82
BsmtFinType1      79
BsmtFinType2      80
Fence           2348
Gar

In [15]:
filling_missing_values(data, 'GarageArea',0)
filling_missing_values(data, 'GarageCars',0)
data['GarageFinish'][(data.GarageFinish.isnull()==True) & (data.GarageCond==0)] =0
data['GarageType'][(data.GarageType.isnull()==True) & (data.GarageCond==0)] =0
data['GarageYrBlt'][(data.GarageYrBlt.isnull()==True) & (data.GarageCond==0)] =0

In [16]:
print(data[['MiscFeature','MiscVal']][(data.MiscFeature=='None') & (data.MiscVal>0)])
data.MiscVal.loc[2550] = 0

print(data[['MiscFeature','MiscVal']][(data.MiscVal==0) & (data.MiscFeature!='None')])
c=data[['MiscFeature','MiscVal']][(data.MiscVal==0) & (data.MiscFeature!='None')].index
data.MiscFeature.loc[c] = 'None'

     MiscFeature  MiscVal
Id                       
2550        None    17000
     MiscFeature  MiscVal
Id                       
874         Othr        0
1201        Shed        0
2432        Shed        0


In [17]:
def inputing(variab):
    y = data[variab]
    data2 = data.drop([variab],axis=1)
    col = data2.columns[data2.isnull().sum()==0]
    data2 = data2[col]
    data2 = pd.get_dummies(data2)
    c_train = y[y.notnull()==True].index
    y_train = y[c_train]
    columny = data2.columns
    X_train = data2[columny].loc[c_train]
    c_test = y[y.notnull()!=True].index
    y_test = y[c_test]
    X_test = data2[columny].loc[c_test]
    #Model
    model = RandomForestClassifier()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    #Filling missing data
    y_pred = pd.Series(y_pred, index=c_test)
    data[variab].loc[c_test] = y_pred.loc[c_test]
    
def inputingnum(variab):
    y = data[variab]
    data2 = data.drop([variab],axis=1)
    col = data2.columns[data2.isnull().sum()==0]
    data2 = data2[col]
    data2 = pd.get_dummies(data2)
    c_train = y[y.notnull()==True].index
    y_train = y[c_train]
    columny = data2.columns
    X_train = data2[columny].loc[c_train]
    c_test = y[y.notnull()!=True].index
    y_test = y[c_test]
    X_test = data2[columny].loc[c_test]
    #Model
    model = RandomForestRegressor()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    #Filling missing data
    y_pred = pd.Series(y_pred, index=c_test)
    data[variab].loc[c_test] = y_pred.loc[c_test]

In [18]:
inputing(variab='Electrical')
inputing(variab='Exterior2nd')
inputing(variab='Exterior1st')
inputing(variab='MasVnrType')
inputing(variab='Functional')
inputing(variab='MSZoning')
inputing(variab='SaleType')
inputing(variab='Alley')
inputing(variab='BsmtExposure')
inputing(variab='BsmtFinType1')
inputing(variab='BsmtFinType2')
inputing(variab='Fence')

inputingnum(variab='KitchenQual')
data['KitchenQual'] = data.KitchenQual.astype(int)
inputingnum(variab='BsmtFullBath')
data['BsmtFullBath'] = data.BsmtFullBath.astype(int)
inputingnum(variab='BsmtHalfBath')
data['BsmtHalfBath'] = data.BsmtHalfBath.astype(int)

inputingnum(variab='TotalBsmtSF')
inputingnum(variab='BsmtFinSF1')
inputingnum(variab='BsmtFinSF2')
inputingnum(variab='MasVnrArea')
inputingnum(variab='BsmtUnfSF')
inputingnum(variab='LotFrontage')

In [19]:
print(data['Utilities'].value_counts())
data  = data.drop(['Utilities'],axis=1)

AllPub    2916
NoSeWa       1
Name: Utilities, dtype: int64


In [20]:
data.columns[data.isnull().sum()>0]

Index(['SalePrice'], dtype='object')

In [21]:
data.describe()

,1stFlrSF,2ndFlrSF,3SsnPorch,BedroomAbvGr,BsmtFinSF1,BsmtFinSF2,BsmtFullBath,BsmtHalfBath,BsmtUnfSF,EnclosedPorch,...,PoolArea,SalePrice,ScreenPorch,TotRmsAbvGrd,TotalBsmtSF,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold,train
count,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,...,2919.000000,1460.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000
mean,1159.581706,336.483727,2.602261,2.860226,441.272011,49.565262,0.429599,0.061322,560.579993,23.098321,...,2.251799,180921.195890,16.062350,6.451524,1051.417266,93.709832,1971.312778,1984.264474,2007.792737,0.500171
std,392.362079,428.701456,25.188169,0.822693,455.606014,169.179104,0.524676,0.245608,439.590889,64.244246,...,35.663946,79442.502883,56.184365,1.569379,441.120498,126.526589,30.291442,20.894344,1.314964,0.500086
min,334.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,34900.000000,0.000000,2.000000,0.000000,0.000000,1872.000000,1950.000000,2006.000000,0.000000
25%,876.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,220.000000,0.000000,...,0.000000,129975.000000,0.000000,5.000000,793.000000,0.000000,1953.500000,1965.000000,2007.000000,0.000000
50%,1082.000000,0.000000,0.000000,3.000000,368.000000,0.000000,0.000000,0.000000,467.000000,0.000000,...,0.000000,163000.000000,0.000000,6.000000,989.000000,0.000000,1973.000000,1993.000000,2008.000000,1.000000
75%,1387.500000,704.000000,0.000000,3.000000,733.000000,0.000000,1.000000,0.000000,805.000000,0.000000,...,0.000000,214000.000000,0.000000,7.000000,1302.000000,168.000000,2001.000000,2004.000000,2009.000000,1.000000
max,5095.000000,2065.000000,508.000000,8.000000,5644.000000,1526.000000,3.000000,2.000000,2336.000000,1012.000000,...,800.000000,755000.000000,576.000000,15.000000,6110.000000,1424.000000,2010.000000,2010.000000,2010.000000,1.000000


In [22]:
def dropping_outliers(data, condition):
    #put condition with with reference to the data table, use brackets and (& |) operators, remember about you can drop observation only from train dataset
    condition_to_drop = data[condition].index
    data = data.drop(condition_to_drop)

In [23]:
dropping_outliers(data, (data.SalePrice<100000) & (data.train==1) & (data.LotFrontage>150))
dropping_outliers(data, (data.LotFrontage>200) & (data.train==1))
dropping_outliers(data, (data.SalePrice>700000) & (data.train==1))
dropping_outliers(data, (data.SalePrice>700000) & (data.train==1))
dropping_outliers(data, (data.LotArea>60000) & (data.train==1))
dropping_outliers(data, (data.MasVnrArea>1450) & (data.train==1))
dropping_outliers(data, (data.BedroomAbvGr==8) & (data.train==1))
dropping_outliers(data, (data.KitchenAbvGr==3) & (data.train==1))
dropping_outliers(data, (data['3SsnPorch']>400) & (data.train==1))
dropping_outliers(data, (data.LotArea>100000) & (data.train==1))
dropping_outliers(data, (data.MasVnrArea>1300) & (data.train==1))
dropping_outliers(data, (data.BsmtFinSF1>2000) & (data.train==1) & (data.SalePrice<300000))
dropping_outliers(data, (data.BsmtFinSF2>200) & (data.SalePrice>350000)  & (data.train==1))
dropping_outliers(data, (data.BedroomAbvGr==8) & (data.train==1))
dropping_outliers(data, (data.KitchenAbvGr==3) & (data.train==1))
dropping_outliers(data, (data.TotRmsAbvGrd==2) & (data.train==1))

In [24]:
print(data['CentralAir'].value_counts())
data['CentralAir'] = pd.Series(np.where(data['CentralAir'].values == 'Y', 1, 0),
          data.index)

Y    2723
N     196
Name: CentralAir, dtype: int64


In [25]:
data['2ndFloor'] = pd.Series(np.where(data['2ndFlrSF'].values == 0, 0, 1),data.index)
data['Floors'] = data['1stFlrSF'] + data['2ndFlrSF']
data = data.drop(['1stFlrSF'],axis=1)
data = data.drop(['2ndFlrSF'],axis=1)
data['TotBath'] = data['FullBath'] + (0.5 * data['HalfBath']) + data['BsmtFullBath'] + (0.5 * data['BsmtHalfBath'])
data['Porch'] = data['OpenPorchSF'] + data['3SsnPorch'] + data['EnclosedPorch'] + data['ScreenPorch']
data['TotalSF'] = data['BsmtFinSF1'] + data['BsmtFinSF2'] + data['Floors'] 
data['Pool'] = pd.Series(np.where(data['PoolArea'].values == 0, 0, 1),data.index)
data['Bsmt'] = pd.Series(np.where(data['TotalBsmtSF'].values == 0, 0, 1),data.index)
data['Garage'] = pd.Series(np.where(data['GarageArea'].values == 0, 0, 1),data.index)
data['Fireplace'] = pd.Series(np.where(data['Fireplaces'].values == 0, 0, 1),data.index)
data['Remod'] = pd.Series(np.where(data['YearBuilt'].values == data['YearRemodAdd'].values, 0, 1),data.index)
data['NewHouse'] = pd.Series(np.where(data['YearBuilt'].values == data['YrSold'].values, 1, 0),data.index)
data['Age'] = data['YrSold'] - data['YearRemodAdd']

In [26]:
c = data[(data['Floors']>4000) & (data.train==1)].index
data = data.drop(c)
c = data[(data['SalePrice']>500000) & (data['TotalSF']<3500) & (data.train==1)].index
data = data.drop(c)

In [27]:
data = data.drop(['PoolQC'],axis=1)
data = data.drop(['GrLivArea'],axis=1)
data = data.drop(['Street'],axis=1)
data = data.drop(['GarageYrBlt'],axis=1)
data = data.drop(['PoolArea'],axis=1)
data = data.drop(['MiscFeature'],axis=1)

In [28]:
Results = pd.DataFrame({'Model': [],'Accuracy Score': []})

In [29]:
data = pd.get_dummies(data)

In [31]:
data.to_csv('./input/data.csv', index = False)

In [34]:
from sklearn.model_selection import train_test_split

trainX, testX, trainY, testY = train_test_split(data[data.SalePrice.isnull()==False].drop('SalePrice',axis=1),data.SalePrice[data.SalePrice.isnull()==False],test_size=0.30, random_state=2019)
trainY = np.log(trainY)

In [33]:
from xgboost.sklearn import XGBRegressor

model = XGBRegressor(learning_rate=0.001,n_estimators=4600,
                                max_depth=7, min_child_weight=0,
                                gamma=0, subsample=0.7,
                                colsample_bytree=0.7,
                                scale_pos_weight=1, seed=27,
                                reg_alpha=0.00006)
model.fit(trainX,trainY)
y_pred = model.predict(testX)
y_pred = np.exp(y_pred)

res = pd.DataFrame({"Model":['XGBoost'],
                    "Accuracy Score": [rmsle(testY, y_pred)]})
Results = Results.append(res)

[13:17:09] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [35]:
from sklearn.tree import DecisionTreeRegressor
model = DecisionTreeRegressor(max_depth=6)
model.fit(trainX,trainY)
y_pred = model.predict(testX)
y_pred = np.exp(y_pred)

print(rmsle(testY, y_pred))

res = pd.DataFrame({"Model":['Decision Tree'],
                    "Accuracy Score": [rmsle(testY, y_pred)]})
Results = Results.append(res)

0.1909953120464487


In [36]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=1500,
                                max_depth=6)
model.fit(trainX,trainY)
y_pred = model.predict(testX)
y_pred = np.exp(y_pred)
print(rmsle(testY, y_pred))

res = pd.DataFrame({"Model":['Random Forest'],
                    "Accuracy Score": [rmsle(testY, y_pred)]})
Results = Results.append(res)

0.14619274259730536


In [37]:
from sklearn.linear_model import Lasso
model = Lasso(alpha=0.0005)

model.fit(trainX,trainY)
y_pred = model.predict(testX)
y_pred = np.exp(y_pred)
print(rmsle(testY, y_pred))

res = pd.DataFrame({"Model":['LASSO'],
                    "Accuracy Score": [rmsle(testY, y_pred)]})
Results = Results.append(res)

0.11467804973961397


In [38]:
import statsmodels.api as sm
X2 = sm.add_constant(trainX)
o=0
for i in X2.columns:
    o+=1
    print(o)
    model = sm.OLS(trainY, X2.astype(float))
    model = model.fit()
    p_values = pd.DataFrame(model.pvalues)
    p_values = p_values.sort_values(by=0, ascending=False)
    if float(p_values.loc[p_values.index[0]])>=0.05:
        X2=X2.drop(p_values.index[0],axis=1)
    else:
        break

kolumny = X2.columns
testX = sm.add_constant(testX)
testX = testX[kolumny]

y_pred = model.predict(testX)
y_pred = np.exp(y_pred)


res = pd.DataFrame({"Model":['Stepwise Regression'],
                    "Accuracy Score": [rmsle(testY, y_pred)]})
Results = Results.append(res)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162


In [40]:
# from sklearn.linear_model import Ridge
# model = Ridge(alpha=0.0005)

# model.fit(trainX,trainY)
# y_pred = model.predict(testX)
# y_pred = np.exp(y_pred)
# print(rmsle(testY, y_pred))

# res = pd.DataFrame({"Model":['Ridge'],
#                     "Accuracy Score": [rmsle(testY, y_pred)]})
# Results = Results.append(res)

In [43]:
# from sklearn.linear_model import Lasso
# model = Lasso(alpha=0)

# model.fit(trainX,trainY)
# y_pred = model.predict(testX)
# y_pred = np.exp(y_pred)
# print(rmsle(testY, y_pred))

# res = pd.DataFrame({"Model":['Linear Regression'],
#                     "Accuracy Score": [rmsle(testY, y_pred)]})
# Results = Results.append(res)

In [44]:
trainX = data[data.SalePrice.isnull()==False].drop(['SalePrice','train'],axis=1)
trainY = data.SalePrice[data.SalePrice.isnull()==False]
testX = data[data.SalePrice.isnull()==True].drop(['SalePrice','train'],axis=1)
trainY = np.log(trainY)
model = Lasso(alpha=0.0005)
model.fit(trainX, trainY)
test = data[data.train==0]
test['SalePrice'] = model.predict(testX)
test['SalePrice'] = np.exp(test['SalePrice'] )
test = test.reset_index()
test[['Id','SalePrice']].to_csv("submissionLASSO.csv",index=False)
print("done1")

done1


In [46]:
trainX = data[data['SalePrice'].isnull()==False].drop(['SalePrice','train'],axis=1)
trainY = data.SalePrice[data.SalePrice.isnull()==False]
testX = data[data.SalePrice.isnull()==True].drop(['SalePrice','train'],axis=1)

trainY = np.log(trainY)

In [50]:
train = trainX
train['SalePrice'] = np.exp(trainY)

test = testX

In [52]:
train.to_csv('./input/train2.csv', index = False)
test.to_csv('./input/test2.csv', index = False)